### DB 연결

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
import time
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine

In [2]:
#pip freeze > requirements.txt

In [5]:
import psycopg2
from sshtunnel import SSHTunnelForwarder


try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


### 테이블확인

In [20]:
import psycopg2
import os

# 사용 가능한 모든 스키마 조회
curs.execute("SELECT DISTINCT schemaname FROM pg_tables WHERE schemaname NOT IN ('pg_catalog', 'information_schema');")
schemanames = curs.fetchall()
schemanames


[('mimiciv_hosp',), ('mimiciv_derived',), ('mimiciv_icu',), ('mimiciv_note',)]

In [22]:
#어떤 스키마에 어떤 테이블이 있는지 확인
curs.execute("select * from PG_TABLEs;")
rows = curs.fetchall( ) #결과가지고옴

for row in rows:
    print(row)

# 파일에 결과 저장
with open("db_tables.txt", "w") as file:
    for row in rows:
        # 각 row의 값들을 문자열로 변환하고 콤마로 구분하여 한 줄로 만듬
        line = ", ".join(map(str, row))
        file.write(line + "\n")  # 파일에 쓰고, 다음 줄로 넘어감


('mimiciv_hosp', 'diagnoses_icd', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'labevents', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'd_icd_diagnoses', 'mai_admin', None, True, False, False, False)
('mimiciv_hosp', 'd_icd_procedures', 'mai_admin', None, True, False, False, False)
('mimiciv_hosp', 'emar', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'hcpcsevents', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'd_labitems', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'pharmacy', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'omr', 'mai_admin', None, False, False, False, False)
('mimiciv_hosp', 'poe_detail', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'poe', 'mai_admin', None, True, False, True, False)
('pg_catalog', 'pg_statistic', 'postgres', None, True, False, False, False)
('pg_catalog', 'pg_type', 'postgres', None, True, False, False, False)
('mimiciv_hosp', 'pro

In [ ]:
# 어떤 변수들부터 뽑아볼까?
# ventilator mode
# Peak pressure(cmH2O)
# Plateau pressure(cmH2O)
# Mean pressure(cmH2O)
# PEEP ((cmH2O))
# Tidal volume (mL)
# FiO2 (%)
# Phigh(cmH2O)
# Plow(cmH2O)
# Thigh(sec)
# Tlow(sec)

In [23]:
curs.execute("select * from mimiciv_derived.ventilator_setting;")
columns_name = [desc[0] for desc in curs.description]
ventilator_setting = pd.DataFrame(curs.fetchall(), columns=columns_name) 
ventilator_setting

,subject_id,stay_id,charttime,respiratory_rate_set,respiratory_rate_total,respiratory_rate_spontaneous,minute_volume,tidal_volume_set,tidal_volume_observed,tidal_volume_spontaneous,plateau_pressure,peep,fio2,flow_rate,ventilator_mode,ventilator_mode_hamilton,ventilator_type
0,10000980,39765666,2189-06-27 09:00:00,NaN,27.0,27.0,11.2,NaN,393.0,528.0,NaN,5.0,NaN,NaN,CPAP/PSV,None,Drager
1,10000980,39765666,2189-06-27 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,Standby,None,None
2,10001884,37510196,2131-01-11 04:00:00,20.0,20.0,0.0,9.0,450.0,479.0,NaN,24.0,5.0,100.0,39.9,None,APV (cmv),Hamilton
3,10001884,37510196,2131-01-11 07:00:00,20.0,20.0,0.0,8.0,400.0,408.0,NaN,22.0,5.1,50.0,35.6,None,APV (cmv),Hamilton
4,10001884,37510196,2131-01-11 11:00:00,20.0,20.0,0.0,8.3,400.0,430.0,NaN,23.0,5.2,50.0,32.3,None,APV (cmv),Hamilton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006122,19999987,36195440,2145-11-03 22:37:00,NaN,29.0,29.0,8.8,NaN,429.0,641.0,NaN,10.0,50.0,NaN,CPAP/PSV,None,None
1006123,19999987,36195440,2145-11-04 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standby,None,None
1006124,19999987,36195440,2145-11-04 07:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,None,None,None
1006125,19999987,36195440,2145-11-04 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN,None,None,None


In [24]:
#ventilator_setting.to_csv('../outputs/ventilator_setting.csv', index=False, encoding='utf-8-sig')

In [25]:
ventilator_setting = pd.read_csv('../outputs/ventilator_setting.csv', parse_dates=['charttime'])

In [27]:
ventilator_setting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006127 entries, 0 to 1006126
Data columns (total 17 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   subject_id                    1006127 non-null  int64         
 1   stay_id                       1006127 non-null  int64         
 2   charttime                     1006127 non-null  datetime64[ns]
 3   respiratory_rate_set          342289 non-null   float64       
 4   respiratory_rate_total        550398 non-null   float64       
 5   respiratory_rate_spontaneous  592180 non-null   float64       
 6   minute_volume                 607284 non-null   float64       
 7   tidal_volume_set              327557 non-null   float64       
 8   tidal_volume_observed         611925 non-null   float64       
 9   tidal_volume_spontaneous      321855 non-null   float64       
 10  plateau_pressure              217024 non-null   float64       
 11

In [28]:
curs.execute("select * from mimiciv_derived.ventilation;")
columns_name = [desc[0] for desc in curs.description]
ventilation = pd.DataFrame(curs.fetchall(), columns=columns_name) 
ventilation

,stay_id,starttime,endtime,ventilation_status
0,30000153,2174-09-29 12:01:00,2174-09-29 20:00:00,InvasiveVent
1,30000153,2174-09-29 20:00:00,2174-10-01 00:57:00,SupplementalOxygen
2,30000213,2162-06-21 05:45:00,2162-06-21 17:35:00,InvasiveVent
3,30000484,2136-01-14 18:46:00,2136-01-17 04:00:00,SupplementalOxygen
4,30000646,2194-04-29 01:40:00,2194-05-03 16:00:00,SupplementalOxygen
...,...,...,...,...
109195,39999230,2147-09-07 13:00:00,2147-09-09 17:00:00,SupplementalOxygen
109196,39999301,2111-08-18 16:11:00,2111-08-19 08:00:00,SupplementalOxygen
109197,39999384,2158-05-24 22:00:00,2158-05-25 04:00:00,SupplementalOxygen
109198,39999552,2186-07-17 16:00:00,2186-07-17 22:00:00,InvasiveVent


In [30]:
#ventilation.to_csv('../outputs/ventilation.csv', index=False, encoding='utf-8-sig')

In [32]:
ventilation = pd.read_csv('../outputs/ventilation.csv', parse_dates=['starttime','endtime'])

In [33]:
ventilation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109200 entries, 0 to 109199
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   stay_id             109200 non-null  int64         
 1   starttime           109200 non-null  datetime64[ns]
 2   endtime             109200 non-null  datetime64[ns]
 3   ventilation_status  109047 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 3.3+ MB


In [ ]:
d_items[d_items['label'].str.contains('Blood Pressure')] 
#itemid 220050 : ABPs (arterial blood pressure systolic), 220051 : ABPm (arterial blood pressure mean), 220052 : ABPd (arterial blood pressure diastolic) #90~140
#itemid 220179 : NBP (non-invasive blood pressure systolic), 220180 : NBPm (non-invasive blood pressure mean), 220181 : NBPd (non-invasive blood pressure diastolic) #60~90
#-----------------------------------
#itemid 224167 Manual BP systolic left, 227242 Manual BP systolic right
#itemid 224643 Manual BP diastolic left, 227243 Manual BP diastolic right

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
6,220050,Arterial Blood Pressure systolic,ABPs,chartevents,Routine Vital Signs,mmHg,Numeric,90.0,140.0
7,220051,Arterial Blood Pressure diastolic,ABPd,chartevents,Routine Vital Signs,mmHg,Numeric,60.0,90.0
8,220052,Arterial Blood Pressure mean,ABPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
9,220056,Arterial Blood Pressure Alarm - Low,ABP Alarm - Low,chartevents,Alarms,mmHg,Numeric,NaN,NaN
10,220058,Arterial Blood Pressure Alarm - High,ABP Alarm - High,chartevents,Alarms,mmHg,Numeric,NaN,NaN
24,220179,Non Invasive Blood Pressure systolic,NBPs,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
25,220180,Non Invasive Blood Pressure diastolic,NBPd,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
26,220181,Non Invasive Blood Pressure mean,NBPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
331,223751,Non-Invasive Blood Pressure Alarm - High,NBP Alarm - High,chartevents,Alarms,mmHg,Numeric,NaN,NaN
332,223752,Non-Invasive Blood Pressure Alarm - Low,NBP Alarm - Low,chartevents,Alarms,mmHg,Numeric,NaN,NaN


In [ ]:
#chartevents 에서 220050, 220051, 220052, 220179, 220180, 220181, 224167, 227242, 224643, 227243 를 찾아야 함

curs.execute("select * from mimiciv_icu.chartevents where itemid in (220050, 220051, 220052, 220179, 220180, 220181, 224167, 227242, 224643, 227243);")
columns_name = [desc[0] for desc in curs.description]
bp_all = pd.DataFrame(curs.fetchall(), columns=columns_name) 
bp_all


MemoryError: Unable to allocate 870. MiB for an array with shape (6, 19002628) and data type int64

In [ ]:
bp_all.to_csv('../outputs/bp_all.csv', index=False, encoding='utf-8-sig')

In [ ]:
bp_all = pd.read_csv('../outputs/bp_all.csv', parse_dates=['charttime'])

In [ ]:

# Remember to close the SSH tunnel
tunnel.close()